In [ ]:
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, BatchNormalization, Dropout,LSTM
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Attention,
                                     Flatten, Reshape, UpSampling3D, MaxPool3D, AveragePooling2D)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import TimeDistributed, GRU, InputLayer, Softmax
from tensorflow.keras import Model
early_stopping = EarlyStopping(patience=30)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

## data

In [ ]:
import pickle
with open('D:\\py\\deeppy\\sst_data4.pickle','rb') as file:
    sst_data = pickle.load(file)

In [ ]:
minidata = sst_data[:,200:400,250:450]

In [ ]:
minval = np.min(np.where(minidata>0,minidata,5000))
maxval = np.max(minidata)

In [ ]:
minidata = (minidata+0.000001-minval)/(maxval+0.000001-minval)

In [ ]:
minidata = np.where(minidata>0,minidata,0)

In [ ]:
west = sst_data[:,275:285,301:311]
east = sst_data[:,275:285,396:406]
south = sst_data[:,322:332,349:359]

In [ ]:
eastdiff = np.max(east)-np.min(east)
eastnor = (east-np.min(east))/(np.max(east) - np.min(east))

In [ ]:
diff = (maxval+0.000001-minval)

In [ ]:
testdata = minidata[:,150:160,40:50]